# Example 3 — Vasp Solid electrolyte 3D example

Here we will look at a small example of looking at the density of fluoride atoms confined in a solid electrolyte BaSnF4 taken from a previous study (https://pubs.acs.org/doi/full/10.1021/jacs.3c08232). This uses the vasp trajectory state and the write out function from the gaussian cube file, and is the main materials science use case. At the moment the method only works for cubic and orthorombic cells.

In [1]:
from revelsMD import trajectory_states, revels_3D
from tqdm import tqdm

We start by creating a list of vaspruns to load into the trajectory state.

In [5]:
vaspruns=[]
for i in tqdm(range(1,11)):
    vaspruns.append(f'../../../examples/example_3_BaSnF4/r{i}/vasprun.xml')

100%|██████████| 10/10 [00:00<00:00, 343795.41it/s]


The vapsrun trajectory state can be generated from the list of files, but care must be taken that forces have been stored in vaspruns.

In [6]:
TS=trajectory_states.VaspTrajectoryState(vaspruns)

Once the TS is loaded we can calculate a density grid. This trajectory is taken from this paper and details of settings can be found in the methods (https://pubs.acs.org/doi/full/10.1021/jacs.3c08232).

In [7]:
GS=revels_3D.Revels3D.GridState(TS,'number',600,nbins=250)
GS.make_force_grid(TS,'F')
GS.get_real_density()

100%|██████████| 14999/14999 [00:03<00:00, 4946.55it/s]


This is great but for the smoothest density we want a linear combination. In order to do this we need to decide the number of sections to divide the frames into to calculate the variance and covariance between for calculating the linear combination. Here we calculate variance between frames by using the same number of sections as frames; this method hasnt been validated in the literature and its use is an extension for the reader. It is best to use 1 frame sections as this method is closest to the method previousely tested in the linear combination paper (https://pubs.aip.org/aip/jcp/article/154/19/191101/565890).

In [8]:
NGS= GS.get_lambda(TS,14999)

100%|██████████| 14999/14999 [9:48:40<00:00,  2.35s/it]  


In [9]:
NGS.write_to_cube(TS.Vasprun.start,NGS.optimal_density,'F_density.cube')